<h1><center>Transition matrix whole week supermarket</center></h1>

In [24]:
#load necessary libraries 
import pandas as pd
import numpy as np

In [25]:
#load necessary data files 
monday=pd.read_csv('data/monday.csv', parse_dates=True, index_col=0, sep=';')
tuesday=pd.read_csv('data/tuesday.csv', parse_dates=True, index_col=0, sep=';')
wednesday=pd.read_csv('data/wednesday.csv', parse_dates=True, index_col=0, sep=';')
thursday=pd.read_csv('data/thursday.csv', parse_dates=True, index_col=0, sep=';')
friday=pd.read_csv('data/friday.csv', parse_dates=True, index_col=0, sep=';')

In [26]:
#reindex time for every minute of the working day and forwrd fill row with info from row above 
#based on: https://stackoverflow.com/questions/32275540/pandas-reindex-dates-in-groupby
def reindex_by_date(df):
    dates=pd.date_range(df.index.min(), df.index.max(), freq='T')
    return df.reindex(dates).ffill()

In [27]:
monday1=monday.groupby('customer_no').apply(reindex_by_date).reset_index(0, drop=True)
tuesday1=tuesday.groupby('customer_no').apply(reindex_by_date).reset_index(0, drop=True)
wednesday1=wednesday.groupby('customer_no').apply(reindex_by_date).reset_index(0, drop=True)
thursday1=thursday.groupby('customer_no').apply(reindex_by_date).reset_index(0, drop=True)
friday1=friday.groupby('customer_no').apply(reindex_by_date).reset_index(0, drop=True)

In [28]:
#creates the following station column by shifting location value up by 1 
#baseon on: https://stackoverflow.com/questions/53335567/use-pandas-shift-within-a-group
monday1['next']=monday1.groupby('customer_no')['location'].shift(-1)
tuesday1['next']=tuesday1.groupby('customer_no')['location'].shift(-1)
wednesday1['next']=wednesday1.groupby('customer_no')['location'].shift(-1)
thursday1['next']=thursday1.groupby('customer_no')['location'].shift(-1)
friday1['next']=friday1.groupby('customer_no')['location'].shift(-1)

In [29]:
#combine all day dfs together to form a week df and create boolean 
#row to indicate which aisle is the first for the customer to visit
week=pd.concat([monday1,tuesday1,wednesday1,thursday1,friday1], axis=0)
week['first']=~week['customer_no'].duplicated()
week['weekday']=week.index.day_name()
week.head()

,customer_no,location,next,first,weekday
2019-09-02 07:03:00,1.0,dairy,dairy,True,Monday
2019-09-02 07:04:00,1.0,dairy,checkout,False,Monday
2019-09-02 07:05:00,1.0,checkout,NaN,False,Monday
2019-09-02 07:03:00,2.0,dairy,dairy,True,Monday
2019-09-02 07:04:00,2.0,dairy,dairy,False,Monday


In [30]:
#export week df for simulation script
week.to_csv('week_data.csv')

In [31]:
P=pd.crosstab(week['location'], week['next'], normalize=0)
P

next,checkout,dairy,drinks,fruit,spices
location,,,,,
dairy,0.102759,0.737331,0.058615,0.049887,0.051408
drinks,0.215549,0.010901,0.598622,0.087927,0.087001
fruit,0.201165,0.095977,0.054878,0.597276,0.050705
spices,0.150008,0.193368,0.163239,0.091025,0.402359


In [32]:
#export the transition matrix for simulation script 
P.to_csv('transition_matrix.csv')